## MIDI Generator Live & Static in Colab
This notebook lets you choose a style workflow or live mode and generate MIDI files directly in Colab.

In [1]:
!pip install -q pretty_midi mido python-rtmidi ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.9 MB/s eta 0:00:00


In [2]:

import json
import os
import time
import random
import pretty_midi
import mido
import ipywidgets as widgets
from IPython.display import display
from pretty_midi import note_name_to_number

# Load configs (upload your config files to /content)
cfg_path = '/content/style_workflow_config.json'
spec_path = '/content/spectrotone_config.json'
if not os.path.exists(cfg_path) or not os.path.exists(spec_path):
    print("Please upload style_workflow_config.json and spectrotone_config.json to /content folder")
else:
    with open(cfg_path) as f:
        cfg = json.load(f)
    with open(spec_path) as f:
        spec_cfg = json.load(f)
    live_rules = cfg.get('live_rules', {})
    style_cfg = cfg['styles']
    print("Configs loaded successfully.")


Configs loaded successfully.


In [3]:

# Style selection
style_dropdown = widgets.Dropdown(options=[(s['name'], s) for s in style_cfg], description='Style:')
sub_dropdown = widgets.Dropdown(description='Substyle:')
bpm_slider = widgets.IntSlider(description='BPM:', min=60, max=200, value=100)
inst_select = widgets.SelectMultiple(description='Instruments:', options=[])
key_dropdown = widgets.Dropdown(options=cfg['keys_modes']['keys'], description='Key:')
mode_dropdown = widgets.Dropdown(options=cfg['keys_modes']['modes'], description='Mode:')
octave_slider = widgets.IntSlider(description='Octave:', min=3, max=5, value=4)
live_toggle = widgets.ToggleButtons(options=[('Static','static'), ('Live','live')], description='Mode:')

def update_subdropdown(*args):
    style = style_dropdown.value
    sub_dropdown.options = [(sub['name'], sub) for sub in style['substyles']]

def update_inst_select(*args):
    sub = sub_dropdown.value
    inst_select.options = sub['instruments']

style_dropdown.observe(update_subdropdown, 'value')
sub_dropdown.observe(update_inst_select, 'value')

display(style_dropdown, sub_dropdown, bpm_slider, inst_select, key_dropdown, mode_dropdown, octave_slider, live_toggle)


Dropdown(description='Style:', options=(('Reggae', {'id': 'reggae', 'name': 'Reggae', 'substyles': [{'id': 'sk…

Dropdown(description='Substyle:', options=(), value=None)

IntSlider(value=100, description='BPM:', max=200, min=60)

SelectMultiple(description='Instruments:', options=(), value=())

Dropdown(description='Key:', options=('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'), value=…

Dropdown(description='Mode:', options=('Ionian', 'Dorian', 'Phrygian', 'Lydian', 'Mixolydian', 'Aeolian', 'Loc…

IntSlider(value=4, description='Octave:', max=5, min=3)

ToggleButtons(description='Mode:', options=(('Static', 'static'), ('Live', 'live')), value='static')

In [5]:

# Output folder
output_path = '/content/output'
os.makedirs(output_path, exist_ok=True)

button = widgets.Button(description='Generate MIDI')
out = widgets.Output()
display(button, out)

def on_button_clicked(b):
    with out:
        out.clear_output()
        style = style_dropdown.value
        sub = sub_dropdown.value
        bpm = bpm_slider.value
        chosen = list(inst_select.value)
        key = key_dropdown.value + str(octave_slider.value)
        mode = mode_dropdown.value
        print(f"Generating {live_toggle.value} MIDI for {style['name']}→{sub['name']} at {bpm} BPM, key {key} {mode}")
        template = {
            'instruments': {i: None for i in chosen},
            'drum_pattern': live_rules.get('drum_pattern', {}),
            'chord_progression': sub.get('chord_progression', ['I','IV','V','I']),
            'length_bars': sub.get('length_bars', 16)
        }
        key_root = note_name_to_number(key)
        if live_toggle.value == 'static':
            from midi_generator import generate_midi
            out_file = os.path.join(output_path, f"static_{style['id']}_{sub['id']}_{bpm}_{key}{mode}.mid")
            generate_midi(template, bpm, key_root, mode, chosen, out_file)
            print(f"Saved static MIDI to {out_file}")
        else:
            from live_randomizer import generate_live_grid
            grid = generate_live_grid(template, live_rules, key_root, mode, bpm)
            midi = pretty_midi.PrettyMIDI(initial_tempo=bpm)
            beat_dur = 60.0/bpm/4
            # Drums
            dr = pretty_midi.Instrument(is_drum=True, name='Drums')
            for role, beats in live_rules.get('drum_pattern',{}).items():
                for b in beats:
                    m = (b-1)//16; t = (b-1)%16
                    if m < live_rules['measures']:
                        time0 = (m*16 + t) * beat_dur
                        dr.notes.append(pretty_midi.Note(100, pretty_midi.gm.drum_map.get(role,36), time0, time0+0.05))
            midi.instruments.append(dr)
            # Others
            for inst in chosen:
                if inst=='Drums': continue
                inst_obj = pretty_midi.Instrument(program=0, name=inst)
                for m, row in enumerate(grid[inst]):
                    for t, on in enumerate(row):
                        if on:
                            t0 = (m*16 + t)*beat_dur
                            inst_obj.notes.append(pretty_midi.Note(100, key_root, t0, t0+beat_dur))
                midi.instruments.append(inst_obj)
            out_file = os.path.join(output_path, f"live_{style['id']}_{sub['id']}_{bpm}_{key}{mode}.mid")
            midi.write(out_file)
            print(f"Saved live MIDI to {out_file}")

button.on_click(on_button_clicked)


Button(description='Generate MIDI', style=ButtonStyle())

Output()